In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['PYTHONHASHSEED'] = str(50)


In [29]:
np.random.seed(11)
np.random.rand(10) / 2

array([0.09013484, 0.00973762, 0.23160926, 0.36246696, 0.2101018 ,
       0.24271355, 0.00639041, 0.2436858 , 0.47090333, 0.42539754])

In [2]:
import math

def bin_append(set_ree, count, pset_size):
    intersect_val = 1
    subset = []

    while(intersect_val < pset_size):
        if count & intersect_val:
            index = int(math.log(intersect_val, 2))
            subset.append(set_ree[index])
        
        intersect_val *= 2

    return subset

def gen_power_set(set_ree):
    power = []
    count = 0

    pset_size = int(2 ** len(set_ree))

    for i in range(pset_size):
        power.append(bin_append(set_ree, count, pset_size))
        count = count + 1

    power.sort(key = lambda x : len(x))
    return power

In [3]:
gen_power_set([0, 1, 2])

[[], [0], [1], [2], [0, 1], [0, 2], [1, 2], [0, 1, 2]]